Hardware Preview

In [1]:
!lscpu
# Expect Intel Xeon Chip @2.0Ghz with 2 cores 4 threads, 2xT4 wont give you this much CPU power, but more GPU power in predicting samples.
!nvidia-smi 
# nVidia P100 expected, 2xT4 requires setup encode_multi_process, read it here https://www.sbert.net/docs/package_reference/SentenceTransformer.html
!python -V 
# Python 3.7.12
!cmake --version
# cmake version 3.22.5

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          4
On-line CPU(s) list:             0-3
Thread(s) per core:              2
Core(s) per socket:              2
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                        0
CPU MHz:                         2199.998
BogoMIPS:                        4399.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       64 KiB
L1i cache:                       64 KiB
L2 cache:                        512 KiB
L3 cache:                        55 MiB
NUMA node0 CPU(s):               0-3
Vulnerability 

In [2]:
!pip install -q pyvi
!pip install -q visen
!pip install -q emoji
!pip install -q pretty-confusion-matrix
!pip install -q Cython
!pip install -q sentence-transformers


# !pip uninstall -q -y lightgbm
# !apt-get -q install -y libboost-all-dev
# !git clone --quiet --recursive https://github.com/Microsoft/LightGBM
!git clone --quiet https://github.com/coccoc/coccoc-tokenizer.git

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytoolconfig 1.2.2 requires tomli>=2.0; python_version < "3.11", but you have tomli 1.2.3 which is incompatible.
autopep8 1.6.0 requires pycodestyle>=2.8.0, but you have pycodestyle 2.7.0 which is incompatible.
allennlp 2.10.0 requires protobuf==3.20.0, but you have protobuf 3.19.4 which is incompatible.


In [3]:
!cd coccoc-tokenizer/ && mkdir build
!cd coccoc-tokenizer/ && cd build && cmake -DBUILD_PYTHON=1 ..
!cd coccoc-tokenizer/ && cd build && make install
!cd coccoc-tokenizer/python && python setup.py install

# !cd LightGBM/ && rm -r build
# !cd LightGBM/ && mkdir build
# !cd LightGBM/ && cd build && cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
# !cd LightGBM/ && cd build && make -j$(nproc)
# !cd LightGBM/python-package/;python setup.py install --precompile
# !mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
# !rm -r LightGBM

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /kaggle/working/coccoc-tokenizer/build
Scanning dependencies of target compile_python
[ 12%] Generating python/lib
running install
running build
running build_ext
cythoning CocCocTokenizer.pyx to CocCocTokenizer.cpp
/opt/conda/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_leve

In [4]:
import re
from unicodedata import normalize
from string import punctuation
import visen
import emoji
from CocCocTokenizer import PyTokenizer
tokenizer = PyTokenizer(load_nontone_data=True).word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, log_loss, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
# import lightgbm as lgb
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import gc
from pretty_confusion_matrix import pp_matrix
import torch
from torch import optim
import scipy
import sys

In [5]:
opening_ls = ['[', '{', '⁅', '〈', '⎡', '⎢', '⎣', '⎧', '⎨', '⎩', '❬', '❰', '❲', '❴', '⟦', '⟨', '⟪', '⟬', '⦃', '⦇', '⦉',
              '⦋', '⦍', '⦏', '⦑', '⦓', '⦕', '⦗', '⧼', '⸂', '⸄', '⸉', '⸌', '⸜', '⸢', '⸤', '⸦', '〈', '《', '「', '『',
              '【', '〔', '〖', '〘', '〚', '﹛', '﹝', '［', '｛', '｢', '｣']

closing_ls = [']', '}', '⁆', '〉', '⎤', '⎥', '⎦', '⎫', '⎬', '⎭', '❭', '❱', '❳', '❵', '⟧', '⟩', '⟫', '⟭', '⦄', '⦈', '⦊',
              '⦌', '⦎', '⦐', '⦒', '⦔', '⦖', '⦘', '⧽', '⸃', '⸅', '⸊', '⸍', '⸝', '⸣', '⸥', '⸧', '〉', '》', '」', '』',
              '】', '〕', '〗', '〙', '〛', '﹜', '﹞', '］', '｝', '｣']

opening_brackets = {key: '(' for key in opening_ls}
closing_brackets = {key: ')' for key in closing_ls}

### constant
PUNC = '!\"#$&()*+,-–−./:;=?@[\]^_`{|}~”“`°²ˈ‐ㄧ‛∼’'  # remove <> for number_sym and unknown_sym
NUMBER_SYM = '<NUMBER>'
UNKNOWN_SYM = ' <UNKNUM> '  # mix number with character, ex: 6.23A
VIETNAMESE_CHARS = 'aàảãáạăằẳẵắặâầẩẫấậbcdđeèẻẽéẹêềểễếệfghiìỉĩíịjklmnrstoòỏõóọôồổỗốộơờởỡớợpquùủũúụưừửữứựvwxyỳỷỹýỵz'
re_email = '([a-zA-Z0-9._%+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z]{2,4}(\\.?[a-zA-Z]{2,4})?)'
re_url = '(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
re_url2 = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
re_image = '(tập[\s_]tin|hình|file|image|imagesize).*?(jpg|svg|png|gif|jpeg|ogg|tif|width)'
re_num_and_decimal = '[0-9]*[,.\-]*[0-9]*[,.\-]*[0-9]*[.,\-]*[0-9]*[,.\-]*[0-9]+[.,]?'
re_unknown = '[a-z]+[\d]+[\w]*|[\d]+[a-z]+[\w]*'
re_vnese_txt = r'[^a-z0-9A-ZàáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹýÀÁÃẠẢĂẮẰẲẴẶÂẤẦẨẪẬÈÉẸẺẼÊỀẾỂỄỆĐÌÍĨỈỊÒÓÕỌỎÔỐỒỔỖỘƠỚỜỞỠỢÙÚŨỤỦƯỨỪỬỮỰỲỴỶỸÝ\s|_]'
re_truncate_unknown = '(<UNKNUM>\s*)+'
special_punc = {'”': '"', '': '', "’": "'", "`": "'"}


def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def replace_all(replacer: dict, txt: str) -> str:
    for old, new in replacer.items():
        txt = txt.replace(old, new)
    return txt


def replace_num(text: str) -> str:
    text = re.sub(re_num_and_decimal, UNKNOWN_SYM, text)
    return text


def replace_unknown(text: str) -> str:
    text = re.sub(re_unknown, UNKNOWN_SYM, text)
    return text


def unicode_normalizer(text, forms: list = ['NFKC', 'NKFD', 'NFC', 'NFD']) -> str:
    for form in forms:
        text = normalize(form, text)
    return text


def normalize_bracket(text: str) -> str:
    text = replace_all(opening_brackets, text)
    text = replace_all(closing_brackets, text)
    text = re.sub(r"[\(\[].*?[\)\]]", " ", text)
    return text


def remove_punc(text: str) -> str:
    r = re.compile(r'[\s{}]+'.format(re.escape(PUNC)))
    text = r.split(text)
    return ' '.join(i for i in text if i)


def truncate_unknown(text: str) -> str:
    text = re.sub(re_truncate_unknown, UNKNOWN_SYM, text)
    return text


def mask(text: str) -> str:
    text = str(text)
    text = text.lower()
    text = text.split('\n')[0]
    text = unicode_normalizer(text, ["NFKC", "NFKD", "NFD", "NFC"])
    text = replace_all(special_punc, text)
    text = normalize_bracket(text)
    text = replace_unknown(text)
    text = replace_num(text)
    text = remove_punc(text)
    text = truncate_unknown(text)
    text = re.sub(re_vnese_txt, " ", text)
    text = text.strip()
    return visen.clean_tone(text)

In [6]:
sample_txts = [
    '〖𝔰𝔬𝔪𝔢 𝔣𝔞𝔫𝔠𝔶 𝔭𝔯𝔬𝔡𝔲𝔠𝔱 𝔫𝔞𝔪𝔢〗',
    '𝖘𝖔𝖒𝖊 𝖋𝖆𝖓𝖈𝖞 𝖕𝖗𝖔𝖉𝖚𝖈𝖙 𝖓𝖆𝖒𝖊',
    '❰some product code which will overfit this sample❱ 𝓼𝓸𝓶𝓮 𝓯𝓪𝓷𝓬𝔂 𝓹𝓻𝓸𝓭𝓾𝓬𝓽 𝓷𝓪𝓶𝓮',
    '𝓈𝑜𝓂𝑒 𝒻𝒶𝓃𝒸𝓎 𝓅𝓇𝑜𝒹𝓊𝒸𝓉 𝓃𝒶𝓂𝑒',
    '{𝕤𝕠𝕞𝕖 𝕗𝕒𝕟𝕔𝕪 𝕡𝕣𝕠𝕕𝕦𝕔𝕥 𝕟𝕒𝕞𝕖',
    '☯😝ｓｏｍｅ ｆａｎｃｙ ｐｒｏｄｕｃｔ ｎａｍｅ☯😝',
    '𝐬𝐨𝐦𝐞 𝐟𝐚𝐧𝐜𝐲 𝐩𝐫𝐨𝐝𝐮𝐜𝐭 𝐧𝐚𝐦𝐞',
    '𝘀𝗼𝗺𝗲 𝗳𝗮𝗻𝗰𝘆 𝗽𝗿𝗼𝗱𝘂𝗰𝘁 𝗻𝗮𝗺𝗲',
    '𝘴𝘰𝘮𝘦 𝘧𝘢𝘯𝘤𝘺 𝘱𝘳𝘰𝘥𝘶𝘤𝘵 𝘯𝘢𝘮𝘦',
    '▄▀▄▀▄▀ 𝙨𝙤𝙢𝙚 𝙛𝙖𝙣𝙘𝙮 𝙥𝙧𝙤𝙙𝙪𝙘𝙩 𝙣𝙖𝙢𝙚 ▄▀▄▀▄▀',
    '🌌  🎀 𝚜𝚘𝚖𝚎 𝚏𝚊𝚗𝚌𝚢 𝚙𝚛𝚘𝚍𝚞𝚌𝚝 𝚗𝚊𝚖𝚎🌌  🎀 ',
    '(っ◔◡◔)っ ♥ some fancy product name ♥',
]

for sample in sample_txts:
    print(f'sample \"{sample}\", after normalize: \"{mask(sample)}\"')

sample "〖𝔰𝔬𝔪𝔢 𝔣𝔞𝔫𝔠𝔶 𝔭𝔯𝔬𝔡𝔲𝔠𝔱 𝔫𝔞𝔪𝔢〗", after normalize: ""
sample "𝖘𝖔𝖒𝖊 𝖋𝖆𝖓𝖈𝖞 𝖕𝖗𝖔𝖉𝖚𝖈𝖙 𝖓𝖆𝖒𝖊", after normalize: "some fancy product name"
sample "❰some product code which will overfit this sample❱ 𝓼𝓸𝓶𝓮 𝓯𝓪𝓷𝓬𝔂 𝓹𝓻𝓸𝓭𝓾𝓬𝓽 𝓷𝓪𝓶𝓮", after normalize: "some fancy product name"
sample "𝓈𝑜𝓂𝑒 𝒻𝒶𝓃𝒸𝓎 𝓅𝓇𝑜𝒹𝓊𝒸𝓉 𝓃𝒶𝓂𝑒", after normalize: "some fancy product name"
sample "{𝕤𝕠𝕞𝕖 𝕗𝕒𝕟𝕔𝕪 𝕡𝕣𝕠𝕕𝕦𝕔𝕥 𝕟𝕒𝕞𝕖", after normalize: "some fancy product name"
sample "☯😝ｓｏｍｅ ｆａｎｃｙ ｐｒｏｄｕｃｔ ｎａｍｅ☯😝", after normalize: "some fancy product name"
sample "𝐬𝐨𝐦𝐞 𝐟𝐚𝐧𝐜𝐲 𝐩𝐫𝐨𝐝𝐮𝐜𝐭 𝐧𝐚𝐦𝐞", after normalize: "some fancy product name"
sample "𝘀𝗼𝗺𝗲 𝗳𝗮𝗻𝗰𝘆 𝗽𝗿𝗼𝗱𝘂𝗰𝘁 𝗻𝗮𝗺𝗲", after normalize: "some fancy product name"
sample "𝘴𝘰𝘮𝘦 𝘧𝘢𝘯𝘤𝘺 𝘱𝘳𝘰𝘥𝘶𝘤𝘵 𝘯𝘢𝘮𝘦", after normalize: "some fancy product name"
sample "▄▀▄▀▄▀ 𝙨𝙤𝙢𝙚 𝙛𝙖𝙣𝙘𝙮 𝙥𝙧𝙤𝙙𝙪𝙘𝙩 𝙣𝙖𝙢𝙚 ▄▀▄▀▄▀", after normalize: "some fancy product name"
sample "🌌  🎀 𝚜𝚘𝚖𝚎 𝚏𝚊𝚗𝚌𝚢 𝚙𝚛𝚘𝚍𝚞𝚌𝚝 𝚗𝚊𝚖𝚎🌌  🎀 ", after normalize: "some fancy product name"
sample "(っ◔◡◔)っ ♥ some fancy product name ♥", after normalize: "some fancy product name"


##### Data preparation

In [7]:
stopwords = pd.read_csv('/kaggle/input/data-dl-final/custom_stop_words.csv', header=0, encoding='utf-8')
stopwords = stopwords['words'].tolist()
masked_data = '/kaggle/input/data-dl-final/masked_data.csv'
masked = pd.read_csv(masked_data, encoding='utf8')

In [8]:
masked = masked.sample(frac=1)
masked

,sample,label
340635,nước hoa hồng byphasse lotion tonique face mọi...,Mỹ phẩm
379540,bộ nước rửa mặt <UNKNUM> thành phần thiên nhiên,Mỹ phẩm
847011,lucky <UNKNUM> analog stick joystick axis sens...,unknown
688584,viên uống collagen nature's way beauty collage...,unknown
939931,battlefield v <UNKNUM>,unknown
...,...,...
1031710,ak nút bèo trắng,Thời trang
611916,thanh chắn cửa chắn cầu thang umoo chặn cửa cầ...,Mẹ & Bé
267741,<UNKNUM> sleeping silk night eye mask cover he...,Mỹ phẩm
1065189,quần jogger <UNKNUM> xl,Thời trang


In [9]:
masked['sample'] = masked['sample'].str.replace('<UNKNUM>','')
masked['sample'] = masked['sample'].str.replace(r'\s+', ' ', regex=True)
masked = masked[~(masked['label'] == 'unknown')]
masked.dropna(inplace=True)
masked['label'].value_counts()

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Điện tử - Điện máy    260085
Thời trang            196907
Mỹ phẩm               189248
Mẹ & Bé               177774
Name: label, dtype: int64

In [10]:
vectorizer = CountVectorizer(lowercase=False, ngram_range=(1, 3), stop_words=stopwords, analyzer='word',tokenizer =tokenizer, min_df=0.001)
X = vectorizer.fit_transform(masked['sample'])
X = X.astype(np.float64)
y = masked['label']

def get_name(num):
    convert = {
        0: "Mẹ & Bé",
        1: "Mỹ phẩm",
        2: "Thời trang",
        3: "Điện tử - Điện máy"
    }
    return convert.get(num, 'idk')

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

MODEL1


In [12]:
from sklearn.ensemble import RandomForestClassifier
model1 = RandomForestClassifier(max_depth=9, random_state=0, n_jobs =-1)
model1.fit(X_train, y_train) 
y_pred = model1.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[21145   421  1045 13118]
 [   85 19193   654 17927]
 [ 1062   668 25323 12096]
 [  153   110   277 51526]]
                    precision    recall  f1-score   support

           Mẹ & Bé       0.94      0.59      0.73     35729
           Mỹ phẩm       0.94      0.51      0.66     37859
        Thời trang       0.93      0.65      0.76     39149
Điện tử - Điện máy       0.54      0.99      0.70     52066

          accuracy                           0.71    164803
         macro avg       0.84      0.68      0.71    164803
      weighted avg       0.81      0.71      0.71    164803

0.7110732207544765


In [13]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred1 = dtc.predict(X_test)
print(confusion_matrix(y_test,y_pred1))
print(classification_report(y_test,y_pred1))
print(accuracy_score(y_test, y_pred1))

[[33803   733   650   543]
 [  898 36031   562   368]
 [  342   246 38470    91]
 [  869   512   347 50338]]
                    precision    recall  f1-score   support

           Mẹ & Bé       0.94      0.95      0.94     35729
           Mỹ phẩm       0.96      0.95      0.96     37859
        Thời trang       0.96      0.98      0.97     39149
Điện tử - Điện máy       0.98      0.97      0.97     52066

          accuracy                           0.96    164803
         macro avg       0.96      0.96      0.96    164803
      weighted avg       0.96      0.96      0.96    164803

0.9626159717966299


MODEL2

In [14]:
#import tensorflow as tf
#from keras.models import Model
#from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Flatten
#from tensorflow.keras.optimizers import Adam
#from keras.preprocessing import sequence
#from tensorflow.keras.utils import to_categorical
#from keras.callbacks import EarlyStopping

In [15]:
#def RNN():
    #inputs = Input(name='inputs',shape=1818)
    #layer = Embedding(1000,50,input_length=1818)(inputs)
    #layer = LSTM(64)(layer)
    #layer = Dense(256,name='FC1')(layer)
    #layer = Activation('relu')(layer)
    #layer = Dropout(0.5)(layer)
    #layer = Dense(1,name='out_layer')(layer)
    #layer = Activation('sigmoid')(layer)
    #layer = Flatten()(layer)
    #model = Model(inputs=inputs,outputs=layer)
    #return model
#model2 = RNN()

In [16]:
#adam = tf.keras.optimizers.Adam(learning_rate = 0.001)
#model2.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = adam)
#model2.fit(X_train,y_train,batch_size=128,epochs=20,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
#y_pred2 = model2.predict(X_test)
#print(confusion_matrix(y_test,y_pred2))
#print(classification_report(y_test,y_pred2))
#print(accuracy_score(y_test, y_pred2))

In [17]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
bnb.fit(X_train, y_train)
y_pred2=bnb.predict(X_test)
print(confusion_matrix(y_test,y_pred2))
print(classification_report(y_test,y_pred2))
print(accuracy_score(y_test, y_pred2))

[[32560  1478  1048   643]
 [  620 36106   758   375]
 [  268    86 38708    87]
 [  564   398   338 50766]]
                    precision    recall  f1-score   support

           Mẹ & Bé       0.96      0.91      0.93     35729
           Mỹ phẩm       0.95      0.95      0.95     37859
        Thời trang       0.95      0.99      0.97     39149
Điện tử - Điện máy       0.98      0.98      0.98     52066

          accuracy                           0.96    164803
         macro avg       0.96      0.96      0.96    164803
      weighted avg       0.96      0.96      0.96    164803

0.9595699107419161


MODEL3

In [18]:
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.pipeline import Pipeline
#from sklearn.feature_extraction.text import TfidfTransformer

#nb = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB())])
#nb.fit(X_train, y_train)

#%%time
#from sklearn.metrics import classification_report
#y_pred3 = nb.predict(X_test)
#print(confusion_matrix(y_test,y_pred3))
#print(classification_report(y_test,y_pred3))
#print(accuracy_score(y_test, y_pred3))

In [19]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)
y_pred3=spam_detect_model.predict(X_test)
print(confusion_matrix(y_test,y_pred3))
print(classification_report(y_test,y_pred3))
print(accuracy_score(y_test, y_pred3))

[[33594  1023   270   842]
 [  864 36089   208   698]
 [  425   183 37670   871]
 [  656   320    66 51024]]
                    precision    recall  f1-score   support

           Mẹ & Bé       0.95      0.94      0.94     35729
           Mỹ phẩm       0.96      0.95      0.96     37859
        Thời trang       0.99      0.96      0.97     39149
Điện tử - Điện máy       0.95      0.98      0.97     52066

          accuracy                           0.96    164803
         macro avg       0.96      0.96      0.96    164803
      weighted avg       0.96      0.96      0.96    164803

0.9610079913593806


MODEL4

In [ ]:
from sklearn.svm import SVC

clf = SVC(gamma='auto')
clf.fit(X_train, y_train)
y_pred4 = clf.predict(X_test)
print(confusion_matrix(y_test,y_pred4))
print(classification_report(y_test,y_pred4))
print(accuracy_score(y_test, y_pred4))

MODEL5

In [ ]:
max_len = np.max(X_train.apply(lambda x :len(x)))
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(1000, 126, input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(3, activation='softmax')
])
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])